### verify files

In [3]:
import os  
  
def check_logs(directory, target_string):  
    # 遍历目录中的所有文件  
    for root, dirs, files in os.walk(directory):  
        for file in files:  
            # 只处理.log文件  
            if file.endswith('.log'):  
                file_path = os.path.join(root, file)  
                with open(file_path, 'r', encoding='utf-8') as f:  
                    content = f.read()  
                    # 检查文件中是否包含目标字符串  
                    if target_string not in content:  
                        print(f"{file_path} 不包含 {target_string}")  
  
# 指定目录和目标字符串  
log_directory = '/mnt/lingjiejiang/textual_aesthetics/prm/data/logs'  
target = 'Processing task 250/250'  
  
# 检查日志文件  
check_logs(log_directory, target)  

### Merge files

In [4]:
import os  
import json  
  
# 指定源目录和目标文件路径  
source_dir = "/mnt/lingjiejiang/textual_aesthetics/prm/data/data/prm_results/"  
output_file = "/mnt/lingjiejiang/textual_aesthetics/prm/data/data/merge_results/merged_results.jsonl"  
  
# 获取源目录下所有文件的列表  
file_list = [f for f in os.listdir(source_dir) if f.endswith('.json')]  
  
# 初始化一个空列表，用于存储所有文件的数据  
all_data = []  
  
# 逐个读取文件并将数据追加到all_data列表中  
for file_name in file_list:  
    file_path = os.path.join(source_dir, file_name)  
    with open(file_path, 'r') as file:  
        for line in file:  
            all_data.append(json.loads(line))  
  
# 将合并后的数据写入目标文件  
with open(output_file, 'w') as output_file:  
    for item in all_data:  
        output_file.write(json.dumps(item) + '\n')  
  
print(f"合并完成，共合并了 {len(file_list)} 个文件，结果存储在 {output_file}")  

合并完成，共合并了 48 个文件，结果存储在 <_io.TextIOWrapper name='/mnt/lingjiejiang/textual_aesthetics/prm/data/data/merge_results/merged_results.jsonl' mode='w' encoding='UTF-8'>


In [ ]:
import json  
import re
class Node:  
    def __init__(self, qid, idx, x, y, step, parent=None, next_nodes=None, gt_answer=None, category=None):  
        self.qid = qid  
        self.idx = idx  
        self.x = x  
        self.y = y  
        self.step = step  
        self.parent = parent  
        self.next_nodes = next_nodes if next_nodes is not None else []  
        self.gt_answer = gt_answer  
        self.pred_acc = self.get_acc()  
        self.is_leaf = False  
        self.is_right = False  
        self.label_he = 0  
        self.label_se = 0  
        self.he_list = []  
        self.se_list = []  
        self.category = category  
  
    def get_acc(self):  
        pred = extract_answer(self.y)  
        self.pred = pred  
        if pred == self.gt_answer:  
            return True  
        else:  
            return False  
  
    def get_info(self):  
        return {  
            'qid': self.qid,  
            'idx': self.idx,  
            'x': self.x,  
            'y': self.y,  
            'step': self.step,  
            'parent': self.parent,  
            'next_nodes': self.next_nodes,  
            'pred_acc': self.pred_acc,  
            "label_he": self.label_he,  
            "label_se": self.label_se,  
            "gt_answer": self.gt_answer,  
            "pred_acc": self.pred_acc,  
            "pred": self.pred,  
            "he_list": self.he_list,  
            "se_list": self.se_list,  
            "category": self.category  
        }  
  
def extract_answer(text):  
    pattern = r"answer is[:\s]*\(?(?P<answer>[A-J])\)?"  
    match = re.search(pattern, text, re.IGNORECASE)  
    if match:  
        return match.group("answer")  
    else:  
        return extract_again(text)  
  
def extract_again(text):  
    pattern = r'.*[aA]nswer[:\s]*([A-J])'  
    match = re.search(pattern, text)  
    if match:  
        return match.group(1)  
    else:  
        return extract_final(text)  
  
def extract_final(text):  
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"  
    match = re.search(pattern, text, re.DOTALL)  
    if match:  
        return match.group(0)  
    else:  
        return None  
  
def load_nodes_from_jsonl(file_path):  
    all_nodes = []  
    with open(file_path, 'r') as f:  
        for line in f:  
            if line.strip():  # 跳过空行  
                try:  
                    node_info = json.loads(line.strip())  
                    all_nodes.append(node_info)  
                except json.JSONDecodeError as e:  
                    print(f"Error decoding JSON: {e}")  
    return all_nodes  
  
def get_leaf_nodes_at_step(all_nodes, step):  
    leaf_nodes = []  
    for node in all_nodes:  
        if node.step == step:  
            leaf_info = {  
                'qid': node.qid,  
                'idx': node.idx,  
                "category": node.category,  
                'step': node.step,  
                'question': node.x,  
                'answer': node.y,  
                'hard_estimation': node.he_list,  
                'soft_estimation': node.se_list,  
                "gt_answer": node.gt_answer,  
                "pred": node.pred  
            }  
            leaf_nodes.append(leaf_info)  
    return leaf_nodes  
  
def write_leaf_nodes_to_file(leaf_nodes, output_file):  
    with open(output_file, 'w') as f:  
        for leaf in leaf_nodes:  
            f.write(json.dumps(leaf) + '\n')  
  
# 示例使用  
input_file = '/mnt/lingjiejiang/textual_aesthetics/prm/data/data/prm_tree_path/Meta-Llama-3.1-8B-Instruct_0.8_Meta-Llama-3.1-8B-Instruct3_match1_greedy1_start0_end250.json'  # 替换为你的输入文件路径  
output_file = 'data/decode_step/path_to_output_file.json'  # 替换为你的输出文件路径  
step = 3  # 替换为你想要获取的层级  
  
# 加载节点  
all_nodes = load_nodes_from_jsonl(input_file)  
  
# 获取指定层级的叶子节点  
leaf_nodes = get_leaf_nodes_at_step(all_nodes, step)  
  
# 写入到文件  
write_leaf_nodes_to_file(leaf_nodes, output_file)  

In [5]:
import json  
import re  
  
DELIMITER = "step"  
TARGET = "the answer is"  
  
def extract_answer(text):  
    pattern = r"answer is[:\s]*\(?(?P<answer>[A-J])\)?"  
    match = re.search(pattern, text, re.IGNORECASE)  
    if match:  
        return match.group("answer")  
    else:  
        return extract_again(text)  
  
def extract_again(text):  
    pattern = r'.*[aA]nswer[:\s]*([A-J])'  
    match = re.search(pattern, text)  
    if match:  
        return match.group(1)  
    else:  
        return extract_final(text)  
  
def extract_final(text):  
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"  
    match = re.search(pattern, text, re.DOTALL)  
    if match:  
        return match.group(0)  
    else:  
        return None  
  
class Node:  
    def __init__(self, qid, idx, x, y, step, parent=None, next_nodes=None, gt_answer=None, category=None):  
        self.qid = qid  
        self.idx = idx  
        self.x = x  
        self.y = y  
        self.step = step  
        self.parent = parent  
        self.next_nodes = next_nodes if next_nodes is not None else []   
        self.gt_answer = gt_answer  
        self.pred_acc = self.get_acc()  
        self.is_leaf = False  
        self.is_right = False  
        self.label_he = 0  
        self.label_se = 0  
        self.he_list = []  
        self.se_list = []  
        self.category = category  
  
    def get_info(self):  
        return {  
            'qid': self.qid,  
            'idx': self.idx,  
            'x': self.x,  
            'y': self.y,  
            'step': self.step,  
            'parent': self.parent,  
            'next_nodes': self.next_nodes,  
            'pred_acc': self.pred_acc,  
            "label_he": self.label_he,  
            "label_se": self.label_se,  
            "gt_answer": self.gt_answer,  
            "pred_acc": self.pred_acc,  
            "pred": self.pred,  
            "he_list": self.he_list,  
            "se_list": self.se_list,  
            "category": self.category  
        }  
  
    def get_acc(self):  
        pred = extract_answer(self.y)  
        self.pred = pred  
        return pred == self.gt_answer  
  
    def get_is_leaf(self):  
        return False  
  
    def get_is_right(self, delimiter=DELIMITER):  
        step_pattern = re.compile(re.escape(delimiter), re.IGNORECASE)  
        matches = list(step_pattern.finditer(self.y))  
        return len(matches) == self.step and self.step > 0  
  
def process_line(line, step):  
    data = json.loads(line)  
    qid = list(data.keys())[0]  
    nodes_data = data[qid]  
    all_nodes = []  
    for node_info in nodes_data:  
        node = Node(  
            qid=node_info['qid'],  
            idx=node_info['idx'],  
            x=node_info['x'],  
            y=node_info['y'],  
            step=node_info['step'],  
            parent=node_info['parent'],  
            next_nodes=node_info['next_nodes'],  
            gt_answer=node_info['gt_answer'],  
            category=node_info['category']  
        )  
        all_nodes.append(node)  
  
    leaf_nodes = get_leaf_nodes_at_step(all_nodes, step)  
    return leaf_nodes  
  
def get_leaf_nodes_at_step(all_nodes, step):  
    leaf_nodes = []  
    for node in all_nodes:  
        if node.step == step:  
            leaf_nodes.append(node.get_info())  
    return leaf_nodes  
  
def process_file(input_file, output_file, step):  
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:  
        for line in infile:  
            leaf_nodes = process_line(line, step)  
            for leaf in leaf_nodes:  
                outfile.write(json.dumps(leaf) + '\n')  
  
# Example usage  
input_file = 'data/prm_tree_path/Meta-Llama-3.1-8B-Instruct_0.8_Meta-Llama-3.1-8B-Instruct3_match1_greedy1_start8564_end8567.json'  
output_file = 'data/decode_step/output.json'  
step = 3  # The step level you want to extract  
  
process_file(input_file, output_file, step)  